# Testing OpenRouter models

We will be testing how well various models available on OpenRouter perform extraction of transport related information from text.

We will be using OPENAI style API to access the models.

In [1]:
# first we need to import the basic libraries
# date
from datetime import datetime
now = datetime.now()
print(f"Date: {now}")
# python version
import sys
print(f"Python version: {sys.version}")
from pathlib import Path
import json
# import time for delay
import time
import requests
# print version
print(f"Requests version: {requests.__version__}")

from tqdm import tqdm
# import OpenAI
# let's try using OpenAI API
from openai import OpenAI
# import openai version
from openai import __version__ as openai_version
print(f"OpenAI version: {openai_version}")


# load OPENROUTER_API key from system environment
import os
# openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY_LNB') # we will be using our institutional key
# assert key is not None
assert openrouter_api_key is not None, "OPENROUTER_API_KEY is not set"
print("We have the OPENROUTER_API_KEY should be good to go")

Date: 2025-02-26 14:42:28.997349
Python version: 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
Requests version: 2.32.3
OpenAI version: 1.59.9
We have the OPENROUTER_API_KEY should be good to go


In [2]:
# let's make a request to the OpenRouter API
response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {openrouter_api_key}",
    "HTTP-Referer": "VSAU", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "VSAU", # Optional. Site title for rankings on openrouter.ai.
  },
  data=json.dumps({
    "model": "openai/gpt-3.5-turbo", # Optional
    "messages": [
      {
        "role": "user",
        "content": "What is the meaning of life?"
      }
    ]
    
  })
)
# print the response
print(response.json())

{'id': 'gen-1740573773-jijhJOwEZ9YWyvc8Oo3I', 'provider': 'OpenAI', 'model': 'openai/gpt-3.5-turbo', 'object': 'chat.completion', 'created': 1740573773, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': 'The meaning of life is a complex and deeply personal question that has been pondered by philosophers, theologians, and individuals throughout history. Different people and belief systems offer varying perspectives on the purpose and meaning of existence. Some believe that the meaning of life is to seek happiness and fulfillment, others see it as a quest for spiritual enlightenment or a way to improve society and leave a positive impact on the world. Ultimately, the meaning of life is subjective and may differ for each individual.', 'refusal': None}}], 'system_fingerprint': None, 'usage': {'prompt_tokens': 14, 'completion_tokens': 97, 'total_tokens': 111}}


In [3]:
# print actual content
print(response.json()['choices'][0]['message']['content'])

The meaning of life is a philosophical question that has been debated by humans for centuries. Different religions, cultures, and individuals have their own interpretations of the meaning of life. Some believe that the purpose of life is to seek happiness and fulfillment, others believe it is to serve a higher power or spiritual purpose, while some believe that life has no inherent meaning and it is up to each individual to create their own meaning. Ultimately, the meaning of life is a deeply personal and subjective question that each person must answer for themselves.


In [4]:
# let's try using OpenAI API
# from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openrouter_api_key,
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "VSAU", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "VSAU", # Optional. Site title for rankings on openrouter.ai.
  },
  model="openai/gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

The meaning of life is a philosophical question that has been debated by thinkers and individuals for centuries. Different perspectives and beliefs exist regarding the purpose and meaning of life, with some attributing it to religious or spiritual beliefs, others focusing on personal fulfillment and happiness, and still others viewing it as a subjective and individual concept. Ultimately, the meaning of life may vary from person to person and can be shaped by one's values, beliefs, experiences, and goals.


## Function to use openai API

In [3]:
# let's make a function that calls the OpenAI API, function will have following parameters:
# base_url, api_key, model, messages, headers
base_url = "https://openrouter.ai/api/v1"
api_key = openrouter_api_key
model = "google/gemini-flash-1.5"
model = "google/gemini-2.0-flash-exp:free"
model = "google/gemini-flash-1.5-8b"
model = "google/gemini-2.0-flash-001"
print(f"Using model {model}")
role = "user"
headers = {
  "HTTP-Referer": "VSAU", # Optional. Site URL for rankings on openrouter.ai.
  "X-Title": "VSAU", # Optional. Site title for rankings on openrouter.ai.
}

system_prompt = """You are an expert on Latvian language and transportation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Show all specific land transportation types found in the text as a list.
No other information is needed, just the transportation terms as they appear in the text.
Provide the terms only in Latvian, preserving their original transcription variants as they appear in the text. Latvian Text follows: """


def openai_chat(user_prompt, 
                system_prompt=system_prompt, 
                base_url=base_url,
                api_key=api_key, 
                model=model, 
                headers=headers,
                verbose=True,
                max_retries=10,
                delay=0.5):
  client = OpenAI(
    base_url=base_url,
    api_key=api_key,
  )
  messages = [
        {
        "role": "system",
        "content": system_prompt
        },
        {
        "role": "user",
        "content": user_prompt
        }
    ]
  # if verbose let's show model, system_prompt and start time for request
  if verbose:
    print(f"Using model {model}")
    print(f"System prompt: {system_prompt}")
    start_time = datetime.now()
    print(f"Start time: {start_time}")
  completion = None
  tries = 0
  while (completion is None or completion.choices is None) and tries < max_retries:
    try:
      completion = client.chat.completions.create(
        extra_headers=headers,
        model=model,
        messages=messages
      )
    except Exception as e:
      print(f"Error: {e}")
      print(f"Trying again... in {delay} seconds")
      # delay
      time.sleep(delay)
    tries += 1
    
  if verbose:
    print(f"Time taken: {datetime.now() - start_time}")
  if completion is None or completion.choices is None:
    return f"Failed to get completion after {max_retries} retries using model {model}"
  return completion.choices[0].message.content


Using model google/gemini-2.0-flash-001


In [4]:
# let's test the function with a prompt
prompt = """Pa ceļu gāja cilvēks ar suni, bet viņš bija ļoti noguris, tāpēc viņš nolēma izmantot sabiedrisko transportu. 
Vēlāk viņš sēdēja autobusā un skatījās uz logu.
Šis cilvēks sapņoja par jūru un pludmali, bet viņam bija jābrauc uz darbu.
Cik labī būtu bijis sēdēt laiva, makšķerēt zivis un baudīt sauli!"""
response = openai_chat(prompt)
print(response)

Using model google/gemini-2.0-flash-001
System prompt: You are an expert on Latvian language and transportation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Show all specific land transportation types found in the text as a list.
No other information is needed, just the transportation terms as they appear in the text.
Provide the terms only in Latvian, preserving their original transcription variants as they appear in the text. Latvian Text follows: 
Start time: 2025-02-26 14:45:24.728420
Time taken: 0:00:02.417762
*   autobusā



In [6]:
# let's list all text files in data/docs folder
# data_folder = Path("../data/docs")
# data_folder = Path("../../lnb_lat_sen_rom_releases/lat_sen_rom_2025_01_28")
data_folder = Path("../../lnb_lat_sen_rom_releases/lat_sen_rom_2025_02_04")
# assert folder exists
assert data_folder.exists(), f"Folder {data_folder} does not exist"
                   
# list all files
files = list(data_folder.glob("*.txt"))
# print all files
# how many files do we have?
print(f"Number of files: {len(files)}")


Number of files: 458


In [7]:
# let's load the files into a dictionary with filename stem as key and text as value
# remember to decode the text as utf-8
texts = {}
for file in tqdm(files):
  with open(file, "r", encoding="utf-8") as f:
    texts[file.stem] = f.read()
# how many texts do we have?
print(f"Number of texts: {len(texts)}")
# how many characters do we have in total?
total_chars = sum([len(text) for text in texts.values()])
print(f"Total characters: {total_chars}")
# what is the smallest text?
min_text = min(texts, key=lambda x: len(texts[x]))
print(f"Key for smallest text: {min_text}")
# how many characters does the smallest text have?
min_chars = len(texts[min_text])


100%|██████████| 458/458 [00:06<00:00, 71.28it/s]

Number of texts: 458
Total characters: 191069647
Key for smallest text: VentA_DepuT_1293527


In [8]:
# let's create responses subfolder
responses_folder = Path("../data/responses")
# responses_folder = data_folder / "responses"
# create folder if it does not exist
responses_folder.mkdir(exist_ok=True)
# assert folder exists
assert responses_folder.exists(), f"Folder {responses_folder} does not exist"
# print full path
print(f"Responses folder: {responses_folder}")


Responses folder: ..\data\responses


## List of transportation terms

In [16]:
land_transportation = """'zirgs' 'dzelzceļš' 'ormanis' 'pajūgs' 'kariete' 'kamanas'
'velosipēds' 'automašīna' 'mašīna' 'vilciens' 'lokomotīve' 'tramvajs' 'bānītis' 'Mercedes'"""
# get rid of '
land_transportation = land_transportation.replace("'", "")
# split by space
land_transportation = land_transportation.split()
# print land_transportation
print(land_transportation)

# now let's recreate system_prompt_2 with unique values
# prompt_with_terms = f"""You are an expert on Latvian language and transportation.
# 1. Please identify which keywords similar to terms in given list of land transportaton vehicles are present in the given text.
# 2. List of keywords: {" ".join(land_transportation)}
# 3. Display the similar keywords as a list preserving their original transcription variants as they appear in the text. Text follows:"""

# prompt_with_terms = f"""You are an expert on Latvian language and transportation.
# Analyze the following Latvian document and identify which terms are semantically similar in meaning to one of the following keywords:
# {' '.join(land_transportation)}
# Display only the semantically related terms exactly as found in the text and nothing else. Latvian text follows: """


prompt_with_terms = f"""You are an expert on Latvian language and transportation.
Analyze the following Latvian document and identify which terms are semantically similar in meaning to one of the following keywords:
{' '.join(land_transportation)}
Display only the semantically related noun terms exactly as found in the text and nothing else. 
For example if the text contains the word 'zirgā' which is a form of 'zirgs' you should include 'zirgā' in the response.
For example if the text contains word 'kučieris' which is related to 'zirgs' you should also include 'kučieris' in the response.
For example if the text contains word 'Fords' which is semantically similar to 'Mercedes' you should include 'Fords' in the response.
Latvian text follows: """
print("Prompt with terms:")
print(prompt_with_terms)

['zirgs', 'dzelzceļš', 'ormanis', 'pajūgs', 'kariete', 'kamanas', 'velosipēds', 'automašīna', 'mašīna', 'vilciens', 'lokomotīve', 'tramvajs', 'bānītis', 'Mercedes']
Prompt with terms:
You are an expert on Latvian language and transportation.
Analyze the following Latvian document and identify which terms are semantically similar in meaning to one of the following keywords:
zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis Mercedes
Display only the semantically related noun terms exactly as found in the text and nothing else. 
For example if the text contains the word 'zirgā' which is a form of 'zirgs' you should include 'zirgā' in the response.
For example if the text contains word 'kučieris' which is related to 'zirgs' you should also include 'kučieris' in the response.
For example if the text contains word 'Fords' which is semantically similar to 'Mercedes' you should include 'Fords' in the response.
Latvian text follows:

## Function to call the model multiple times

In [41]:
# let's make a function that will take as parameters the following:
# texts, system_prompt, verbose, delay, max_retries, model, responses_folder
# first we will create a subfolder in responses_folder named YYYY_MM_DD_model
# then we will iterate over all texts and call openai_chat function with text as prompt
# we will save the response in the subfolder with the same name as the text
# we will delay with given delay between requests
# we will retry max_retries times if we get an error

def openai_chat_all(texts, system_prompt, verbose, delay, max_retries, model, responses_folder, responses_folder_suffix="with_terms"):
  # create subfolder in responses_folder
  # let's create model_name from model by replacing / with _
  model_name = model.replace("/", "_")
  model_name = model_name.replace(":", "_")
  subfolder = responses_folder / f"{datetime.now().strftime('%Y_%m_%d')}_{model_name}_{responses_folder_suffix}"
  if verbose:
    print(f"Subfolder: {subfolder}")
  # create subfolder if it does not exist
  subfolder.mkdir(exist_ok=True)
  # assert subfolder exists
  assert subfolder.exists(), f"Subfolder {subfolder} does not exist"
  # iterate over all texts
  for key, value in tqdm(texts.items()):
    if verbose:
      print(f"Processing key: {key}")
    # call openai_chat function with text as prompt
    response = openai_chat(user_prompt=value, 
                           system_prompt=system_prompt, 
                           verbose=verbose, delay=delay, max_retries=max_retries, model=model)
    # save the response in the subfolder with the same name as the text
    with open(subfolder / f"{key}.txt", "w", encoding="utf-8") as f:
      f.write(response)
      # write system prompt
      f.write(f"\n\nSystem prompt:\n {system_prompt}")
    # delay with given delay between requests
    time.sleep(delay)

# let's test on first 10 texts
# openai_chat_all(dict(list(texts.items())[:20]), prompt_with_terms, True, 0.2, 10, model, responses_folder)

In [ ]:
deep_seek_model = "deepseek/deepseek-r1:free"
# let's try 20 texts with deepseek model
openai_chat_all(dict(list(texts.items())[:20]), prompt_with_terms, True, 0.2, 10, deep_seek_model, responses_folder, "with_terms")

In [21]:
# let's try gemini experimental
gemini_exp_model = "google/gemini-2.0-flash-thinking-exp:free"
# let's try 20 texts with gemini experimental model
openai_chat_all(dict(list(texts.items())[:20]), prompt_with_terms, True, 0.2, 10, gemini_exp_model, responses_folder, "with_terms")

Subfolder: ..\data\responses\2025_02_04_google_gemini-2.0-flash-thinking-exp_free_with_terms


  0%|          | 0/20 [00:00<?, ?it/s]

Processing key: AizsV_MilaU_1049452
Using model google/gemini-2.0-flash-thinking-exp:free
System prompt: You are an expert on Latvian language and transportation.
Analyze the following Latvian document and identify which terms are semantically similar in meaning to one of the following keywords:
zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis Mercedes
Display only the semantically related noun terms exactly as found in the text and nothing else. 
For example if the text contains the word 'zirgā' which is a form of 'zirgs' you should include 'zirgā' in the response.
For example if the text contains word 'kučieris' which is related to 'zirgs' you should also include 'kučieris' in the response.
For example if the text contains word 'Fords' which is semantically similar to 'Mercedes' you should include 'Fords' in the response.
Latvian text follows: 
Start time: 2025-02-04 10:39:44.661895
Time taken: 0:00:06.694392


  5%|▌         | 1/20 [00:07<02:27,  7.78s/it]

Processing key: AkurJ_DegoS_771400
Using model google/gemini-2.0-flash-thinking-exp:free
System prompt: You are an expert on Latvian language and transportation.
Analyze the following Latvian document and identify which terms are semantically similar in meaning to one of the following keywords:
zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis Mercedes
Display only the semantically related noun terms exactly as found in the text and nothing else. 
For example if the text contains the word 'zirgā' which is a form of 'zirgs' you should include 'zirgā' in the response.
For example if the text contains word 'kučieris' which is related to 'zirgs' you should also include 'kučieris' in the response.
For example if the text contains word 'Fords' which is semantically similar to 'Mercedes' you should include 'Fords' in the response.
Latvian text follows: 
Start time: 2025-02-04 10:39:52.431621
Time taken: 0:00:04.333805


...

 90%|█████████ | 18/20 [01:38<00:10,  5.08s/it]

Processing key: AustA_Puisk_1047362
Using model google/gemini-2.0-flash-thinking-exp:free
System prompt: You are an expert on Latvian language and transportation.
Analyze the following Latvian document and identify which terms are semantically similar in meaning to one of the following keywords:
zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis Mercedes
Display only the semantically related noun terms exactly as found in the text and nothing else. 
For example if the text contains the word 'zirgā' which is a form of 'zirgs' you should include 'zirgā' in the response.
For example if the text contains word 'kučieris' which is related to 'zirgs' you should also include 'kučieris' in the response.
For example if the text contains word 'Fords' which is semantically similar to 'Mercedes' you should include 'Fords' in the response.
Latvian text follows: 
Start time: 2025-02-04 10:41:22.743696


 95%|█████████▌| 19/20 [01:42<00:04,  4.72s/it]

Time taken: 0:00:02.894614
Processing key: BaloP_DaugU_1051661
Using model google/gemini-2.0-flash-thinking-exp:free
System prompt: You are an expert on Latvian language and transportation.
Analyze the following Latvian document and identify which terms are semantically similar in meaning to one of the following keywords:
zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis Mercedes
Display only the semantically related noun terms exactly as found in the text and nothing else. 
For example if the text contains the word 'zirgā' which is a form of 'zirgs' you should include 'zirgā' in the response.
For example if the text contains word 'kučieris' which is related to 'zirgs' you should also include 'kučieris' in the response.
For example if the text contains word 'Fords' which is semantically similar to 'Mercedes' you should include 'Fords' in the response.
Latvian text follows: 
Start time: 2025-02-04 10:41:26.682479
Time taken:

100%|██████████| 20/20 [01:45<00:00,  5.30s/it]


In [21]:
prompt_no_terms = system_prompt
print("Prompt without terms:")
print(prompt_no_terms)

Prompt without terms:
You are an expert on Latvian language and transportation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Show all specific land transportation types found in the text as a list.
No other information is needed, just the transportation terms as they appear in the text.
Provide the terms only in Latvian, preserving their original transcription variants as they appear in the text. Latvian Text follows: 


In [22]:
# now let's run first 20 texts without terms
openai_chat_all(dict(list(texts.items())[:20]), prompt_no_terms, True, 0.2, 10, model, responses_folder, "no_terms")

Subfolder: ..\data\responses\2025_01_29_google_gemini-flash-1.5-8b_no_terms


  0%|          | 0/20 [00:00<?, ?it/s]

Processing key: AizsV_MilaU_1049452
Using model google/gemini-flash-1.5-8b
System prompt: You are an expert on Latvian language and transportation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Show all specific land transportation types found in the text as a list.
No other information is needed, just the transportation terms as they appear in the text.
Provide the terms only in Latvian, preserving their original transcription variants as they appear in the text. Latvian Text follows: 
Start time: 2025-01-29 10:40:07.210005


  5%|▌         | 1/20 [00:07<02:18,  7.30s/it]

Time taken: 0:00:06.237332
Processing key: AkurJ_DegoS_771400
Using model google/gemini-flash-1.5-8b
System prompt: You are an expert on Latvian language and transportation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Show all specific land transportation types found in the text as a list.
No other information is needed, just the transportation terms as they appear in the text.
Provide the terms only in Latvian, preserving their original transcription variants as they appear in the text. Latvian Text follows: 
Start time: 2025-01-29 10:40:14.479043
Time taken: 0:00:03.775122


...

 90%|█████████ | 18/20 [04:48<00:43, 21.94s/it]

Processing key: AustA_Puisk_1047362
Using model google/gemini-flash-1.5-8b
System prompt: You are an expert on Latvian language and transportation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Show all specific land transportation types found in the text as a list.
No other information is needed, just the transportation terms as they appear in the text.
Provide the terms only in Latvian, preserving their original transcription variants as they appear in the text. Latvian Text follows: 
Start time: 2025-01-29 10:44:55.741725
Time taken: 0:00:15.183847


 95%|█████████▌| 19/20 [05:04<00:20, 20.21s/it]

Processing key: BaloP_DaugU_1051661
Using model google/gemini-flash-1.5-8b
System prompt: You are an expert on Latvian language and transportation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Show all specific land transportation types found in the text as a list.
No other information is needed, just the transportation terms as they appear in the text.
Provide the terms only in Latvian, preserving their original transcription variants as they appear in the text. Latvian Text follows: 
Start time: 2025-01-29 10:45:11.935732
Time taken: 0:00:15.350159


100%|██████████| 20/20 [05:21<00:00, 16.06s/it]


In [23]:
# now let's run rest in non verbose mode
openai_chat_all(dict(list(texts.items())[20:]), prompt_no_terms, False, 0.1, 10, model, responses_folder, "no_terms")

100%|██████████| 433/433 [1:46:06<00:00, 14.70s/it]  


In [27]:
print(prompt_with_terms)

You are an expert on Latvian language and transportation.
1. Please identify which keywords similar to terms in given list of land transportaton vehicles are present in the given text.
2. List of keywords: zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis
3. Display the similar keywords as a list preserving their original transcription variants as they appear in the text. Text follows:


In [28]:
# let's try first 20 texts with terms
openai_chat_all(dict(list(texts.items())[:20]), prompt_with_terms, True, 0.2, 10, model, responses_folder, "with_terms")

Subfolder: ..\data\responses\2025_01_29_google_gemini-flash-1.5-8b_with_terms


  0%|          | 0/20 [00:00<?, ?it/s]

Processing key: AizsV_MilaU_1049452
Using model google/gemini-flash-1.5-8b
System prompt: You are an expert on Latvian language and transportation.
1. Please identify which keywords similar to terms in given list of land transportaton vehicles are present in the given text.
2. List of keywords: zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis
3. Display the similar keywords as a list preserving their original transcription variants as they appear in the text. Text follows:
Start time: 2025-01-29 18:18:31.196224
Time taken: 0:00:08.651312


  5%|▌         | 1/20 [00:09<03:04,  9.72s/it]

Processing key: AkurJ_DegoS_771400
Using model google/gemini-flash-1.5-8b
System prompt: You are an expert on Latvian language and transportation.
1. Please identify which keywords similar to terms in given list of land transportaton vehicles are present in the given text.
2. List of keywords: zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis
3. Display the similar keywords as a list preserving their original transcription variants as they appear in the text. Text follows:
Start time: 2025-01-29 18:18:40.872988


...

 90%|█████████ | 18/20 [06:09<00:47, 23.55s/it]

Processing key: AustA_Puisk_1047362
Using model google/gemini-flash-1.5-8b
System prompt: You are an expert on Latvian language and transportation.
1. Please identify which keywords similar to terms in given list of land transportaton vehicles are present in the given text.
2. List of keywords: zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis
3. Display the similar keywords as a list preserving their original transcription variants as they appear in the text. Text follows:
Start time: 2025-01-29 18:24:40.719144
Time taken: 0:00:15.409495


 95%|█████████▌| 19/20 [06:25<00:21, 21.42s/it]

Processing key: BaloP_DaugU_1051661
Using model google/gemini-flash-1.5-8b
System prompt: You are an expert on Latvian language and transportation.
1. Please identify which keywords similar to terms in given list of land transportaton vehicles are present in the given text.
2. List of keywords: zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis
3. Display the similar keywords as a list preserving their original transcription variants as they appear in the text. Text follows:
Start time: 2025-01-29 18:24:57.140848
Time taken: 0:00:16.578157


100%|██████████| 20/20 [06:43<00:00, 20.18s/it]


In [ ]:
# now let's run rest in non verbose mode
openai_chat_all(dict(list(texts.items())[20:]), prompt_with_terms, False, 0.1, 10, model, responses_folder, "with_terms")

In [ ]:
# let's call the openai_chat function for each text and save response to responses folder using key as filename
for key, text in tqdm(texts.items()):
  response = openai_chat(text, system_prompt=system_prompt_3)
  with open(responses_folder / f"{key}.txt", "w", encoding="utf-8") as f:
    f.write(response)
    time.sleep(0.5)

In [15]:
# let's open last file and read the content using utf-8 encoding
file = files[-1]
with open(file, encoding="utf-8") as f:
  text = f.read()
# first 300 characters
print(text[:300])

Sieviete


Pirmā daļa

Guberņas rentejas ierēdnis Apse pamodās, kad pulkstens aiz sienas ēdamistabā sāka sist. Pusmiegā viņš skaitīja... un nevarēja saskaitīt. Bet tomēr beigās zināja, ka četri. Četri... viņš vairāk reižu domās atkārtoja šo skaitli. Tad vienreiz izrunāja balsī tā, ka pats varēja dzi


In [17]:
# lets test the function with the text
response = openai_chat(text)
print(response)

Using model google/gemini-2.0-flash-exp:free
System prompt: Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Include specific types (e.g., cars, trains, bicycles) as well as broader categories (e.g., public transportation).
Provide the terms in Latvian, preserving their original transcription variants as they appear in the text. Text:
Start time: 2025-01-23 10:22:22.422112
Time taken: 0:00:11.144166
Pēc dotā teksta, zemes transporta līdzekļi ir:

*  zābaki (nav gan transportlīdzeklis, bet ir saistīts ar pārvietošanos pa zemi; zābaki, un arī kājas, bija bieži pieminēti)
*  kamanas (gan vienjūga, gan divjūga kamanas)
*  tramvaja vagons
*  dzelzceļa vilciens (minēts caur sajūtām - "vai sēd dzelzceļa vilcienā")
*  ekipaža (minēts kā nākotnes sapnis: "brauks sava paša eķipažā")
*  automobiļi (nav tieši minēti, bet netieši pieņemti, ka tie varētu pastāvēt, kad runāts par "pazīstamās automobiļu skaņām")
*  riteņi (saistīti ar velobraucēji

In [20]:
print(system_prompt)

Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Include specific types (e.g., cars, trains, bicycles) as well as broader categories (e.g., public transportation).
Provide the terms in Latvian, preserving their original transcription variants as they appear in the text. Text:


In [29]:
system_prompt = """You are an expert on Latvian language and transportation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Show all specific land transportation types found in the text as a list.
No other information is needed, just the transportation terms as they appear in the text.
Provide the terms only in Latvian, preserving their original transcription variants as they appear in the text. Latvian Text follows: """

In [27]:
# lets test the function with the text
response = openai_chat(text, system_prompt=system_prompt)
print(response)

Using model google/gemini-2.0-flash-exp:free
System prompt: You are an expert on Latvian language and transporation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Show all specific land transportation types found in the text as a list.
No other information is needed, just the transportation terms as they appear in the text.
Provide the terms only in Latvian, preserving their original transcription variants as they appear in the text. Latvian Text follows: 
Start time: 2025-01-24 14:23:16.215333
Time taken: 0:00:12.244052
* zābakiem
* gājēji
* braucēji
*  klavieres
* krēsla
*  vilciens
* kamanu
*  šoseju
*  tramvaja
* vagons
* eķipažā
*  dzelzceļa
* slidotavu
* pajūgu
*  pulktiens
*  ormani
* autobuss



In [28]:
transport_lemmas = sorted("""zirgs
dzelzceļš
ormanis
pajūgs
rati
kariete
kamanas
laiva
plosts
burinieks
kuģis
tvaikonis
velosipēds
automašīna
mašīna
vilciens
lokomotīve
tramvajs
bānītis
vāģi
stīmeris
zēģele
fords
Mercedes
""".split())
transport_lemmas

['Mercedes',
 'automašīna',
 'burinieks',
 'bānītis',
 'dzelzceļš',
 'fords',
 'kamanas',
 'kariete',
 'kuģis',
 'laiva',
 'lokomotīve',
 'mašīna',
 'ormanis',
 'pajūgs',
 'plosts',
 'rati',
 'stīmeris',
 'tramvajs',
 'tvaikonis',
 'velosipēds',
 'vilciens',
 'vāģi',
 'zirgs',
 'zēģele']

In [30]:
system_prompt_2 = f"""You are an expert on Latvian language and transportation.
1. Please identify which keywords from the given list of land transportaton vehicles are present in the given text.
2. List of keywords: {" ".join(transport_lemmas)}
3. Identify all other land transportation vehicles in the given text.
4. Provide terms in Latvian as a list, preserving their original spelling variants as they appear in the text."""

In [38]:
# lets test the function with the text
response = openai_chat(text, system_prompt=system_prompt_2)
print(response)

Using model google/gemini-2.0-flash-exp:free
System prompt: You are an expert on Latvian language and transportation.
1. Please identify which keywords from the given list of land transportaton vehicles are present in the given text.
2. List of keywords: Mercedes automašīna burinieks bānītis dzelzceļš fords kamanas kariete kuģis laiva lokomotīve mašīna ormanis pajūgs plosts rati stīmeris tramvajs tvaikonis velosipēds vilciens vāģi zirgs zēģele
3. Identify all other land transportation vehicles in the given text.
4. Provide terms in Latvian as a list, preserving their original spelling variants as they appear in the text.
Start time: 2025-01-24 14:33:04.955773
Time taken: 0:00:06.156933
Okay, I can help with that! Let's break down the text as you requested.

**1. Keywords from the list found in the text:**

From the provided keyword list, the following terms are present in the text:

*   kamanas
*    mašīna
*   zirgs 
*   tramvajs
*   vilciens
*   vāģi

**2. Additional Land Transportati

In [39]:
files

[WindowsPath('../data/docs/JaunJ_Ziema_417458.txt'),
 WindowsPath('../data/docs/PaulM_ProfS_1051757.txt'),
 WindowsPath('../data/docs/UpitA_Sievi_771386.txt'),
 WindowsPath('../data/docs/UpitA_Siev_part_1.txt')]

In [40]:
# let's load second text in profesors variable
profesors = files[1]
with open(profesors, encoding="utf-8") as f:
  text = f.read()
# first 300 characters
print(text[:300])

title: Profesora Sūnas brīnišķīgais eleksīrs
isPartOf:
creator: Paulockis, Miķelis
dateIssued: 1938
publisher: Senatne
publicationPlace: Rīga
firstPublished:
firstEdition: 1938
dateModified:
copyright:
uri: http://dom.lndb.lv/data/obj/1051757
normalization:
ocrCorrection:
verified:



Profesora Sūna


In [41]:
# now let's use pandas to load last csv from csv subfolder
import pandas as pd

csv_folder = Path("../csv")
# assert csv_folder exists
assert csv_folder.exists(), "csv folder does not exist"
# list all files
csv_files = list(csv_folder.glob("*.csv"))
# sort by last modified
csv_files = sorted(csv_files, key=lambda x: x.stat().st_mtime)
# get latest file
csv_file = csv_files[-1]
# read csv file
df = pd.read_csv(csv_file)
# print first 5 rows
df.head()

,original,term,score
0,zirgs,vezums,0.813284
1,zirgs,rats,0.804218
2,zirgs,kumeļš,0.764619
3,zirgs,bēris,0.740647
4,zirgs,zirdziņš,0.732735


In [42]:
# get unique values from first column
unique_values = df.iloc[:, 0].unique()
# print unique values
print(unique_values)

['zirgs' 'dzelzceļš' 'ormanis' 'pajūgs' 'kariete' 'kamanas' 'laiva'
 'plosts' 'burinieks' 'kuģis' 'tvaikonis' 'velosipēds' 'automašīna'
 'mašīna' 'vilciens' 'lokomotīve' 'tramvajs' 'bānītis' 'stīmeris' 'zēģele']


In [50]:
# let's load last first file as text
ziema_file = files[0]
with open(ziema_file, encoding="utf-8") as f:
  text = f.read()
# first 300 characters
print(text[:300])

title: Ziema
isPartOf:
creator: Jaunsudrabiņš, Jānis, 1877-1962
dateIssued: 1925
publisher: Valters un Rapa
publicationPlace: Rīga
firstPublished:
firstEdition: 1925
dateModified: 26.06.2024
copyright:
uri: http://dom.lndb.lv/data/obj/417458
normalization:
ocrCorrection: Jā
verified: 26.06.2024, And


In [51]:
# let's test the function with profesors text and system_prompt_3
response = openai_chat(text, system_prompt=system_prompt_3)
print(response)

Using model google/gemini-2.0-flash-exp:free
System prompt: You are an expert on Latvian language and transportation.
1. Please identify which keywords from the given list of land transportaton vehicles are present in the given text.
2. List of keywords: zirgs dzelzceļš ormanis pajūgs kariete kamanas velosipēds automašīna mašīna vilciens lokomotīve tramvajs bānītis
3. Identify all other land transportation vehicles in the given text.
4. Provide terms in Latvian as a list, preserving their original spelling variants as they appear in the text.
Start time: 2025-01-24 14:57:33.166346
Time taken: 0:00:11.862424
Okay, here's the breakdown of land transportation terms from the provided text, based on your criteria:

**1. Keywords present in the text:**

Based on the provided list of keywords, the following are present in the text:
*   zirgs
*   velosipēds (not directly, but implied when referencing Pakalns and his method for obtaining water)
*   automašīna (not directly, but implied when ref

In [9]:
# now let's try the same prompt but with a different model
# we will use the following model meta-llama/llama-3.3-70b-instruct
model = "meta-llama/llama-3.3-70b-instruct"
response = openai_chat(prompt, model=model)
print(response)

Here's a list of transportation terms in the Latvian language:

1. Auto (Car) - Auto
2. Autoceļš (Highway) - Autoceļš
3. Velosipēds (Bicycle) - Velosipēds
4. Riteņbraucējs (Cyclist) - Riteņbraucējs
5. Būve (Construction) - Būve
6. Dzelzceļš (Railway) - Dzelzceļš
7. Vilciens (Train) - Vilciens
8. Lidmašīna (Airplane) - Lidmašīna
9. Osta (Port) - Osta
10. Kuģis (Ship) - Kuģis
11. Autobuss (Bus) - Autobuss
12. Trolejbuss (Trolleybus) - Trolejbuss
13. Tramvajs (Tram) - Tramvajs
14. Taksi (Taxi) - Taksi
15. Stacija (Station) - Stacija
16. Pietura (Stop) - Pietura
17. Brauciens (Trip) - Brauciens
18. Reiss (Journey) - Ceļojums (more common term for journey)
19. Maršruts (Route) - Maršruts
20. Satiksme (Transportation) - Satiksme

Additionally, here are some common phrases related to transportation:

- "Kā piesāpināt?" (How to get there?) - Kā piesāpināt?
- "Kā braukt?" (How to travel?) - Kā braukt?
- "Cik ilgi?" (How long does it take?) - Cik ilgi?
- "Cik daudz maksā?" (How much does it cost

In [10]:
# let's try the same prompt but with a different model
# we will use the following model meta-llama/llama-3.2-90b-vision-instruct:free 
model = "meta-llama/llama-3.2-90b-vision-instruct:free"
response = openai_chat(prompt, model=model)
print(response)

Here's a list of common transportation terms in Latvian:

1. Transport - transports (transportation)
2. Autobuss - autobus (bus)
3. Viena - vilciens (train)
4. Lidmašīna - lidmašīna (airplane)
5. Kuģis - kuģis (ship)
6. Laiva - laiva (boat)
7. Riteņbraucējs - riteņbraucējs (bicyclist)
8. Velosipēds - velosipēds (bicycle)
9. Motocikls - motocikls (motorcycle)
10. Automobiļi - mašīna (car)
11. Taksi - taksometrs (taxi)
12. Metro - metro (subway)
13. Tramvajs - tramvajs (tram)
14. Elektrovilciens - elektrības vilciens (electric train)
15. Starptautiskā lidosta - starptautiskā lidosta (international airport)
16. Tuksneša mašīna - kutermašīna (jeep)
17. Gājējs - gājējs (pedestrian)
18. Ceļš - ceļš (road)
19. Automaģistrāle - automaģistrāle (highway)
20. Tilts - tilts (bridge).

Note: Latvian language uses its specific characters, but they were not used here for easier search. Example of those would be:
ā (in transportation - transports), 
ē (in ceļš - road),
ī (in lidmašīna - airplane),
ū (

In [11]:
# now let's try with latest GPT-4o model
model = "openai/gpt-4o-2024-11-20"
response = openai_chat(prompt, model=model)
print(response)

Here is a list of transportation-related terms in Latvian:

1. **Automašīna** – Car  
2. **Autobuss** – Bus  
3. **Vilciens** – Train  
4. **Lidmašīna** – Airplane  
5. **Kuģis** – Ship  
6. **Velosipēds** – Bicycle  
7. **Motocikls** – Motorcycle  
8. **Skrejritenis** – Scooter  
9. **Taksometrs (Taksi)** – Taxi  
10. **Kravas automašīna** – Truck  
11. **Laiva** – Boat  
12. **Tramvajs** – Tram  
13. **Trolejbuss** – Trolleybus  
14. **Helikopters** – Helicopter  
15. **Metro** – Subway  
16. **Automaģistrāle** – Highway  
17. **Satiksmes līdzeklis** – Means of transport  
18. **Ceļš** – Road  
19. **Maršruts** – Route  
20. **Stacija** – Station  
21. **Pietura** – Stop  
22. **Autoosta** – Bus station  
23. **Osta** – Port  
24. **Lidosta** – Airport  
25. **Ceļojums** – Journey  
26. **Biļete** – Ticket  
27. **Ātrums** – Speed  
28. **Degviela** – Fuel  
29. **Pasažieris** – Passenger  
30. **Vadītājs** – Driver  
31. **Stūre** – Steering wheel  
32. **Bagāža** – Luggage  
33. **

In [12]:
prompt

'Provide a list of transportation terms in Latvian language'

In [21]:
model

'google/gemini-flash-1.5'

In [25]:
# let's get list of all txt files in data docs folder
from pathlib import Path
text_files = list(Path("../data/docs").rglob("*.txt"))
print(f"We have {len(text_files)} text files in the data/docs folder")
# first one
print(f"Name of first file: {text_files[0]}")
# 2nd one
print(f"Name of second file: {text_files[1]}")
# last one 
print(f"Name of last file: {text_files[-1]}")

We have 4 text files in the data/docs folder
Name of first file: ..\data\docs\JaunJ_Ziema_417458.txt
Name of second file: ..\data\docs\PaulM_ProfS_1051757.txt
Name of last file: ..\data\docs\UpitA_Siev_part_1.txt


In [18]:
# let's read text of last file using utf-8 encoding
with open(text_files[-1], "r", encoding="utf-8") as file:
  text = file.read()
# how many characters in the text
print(f"Number of characters in the text: {len(text)}")

Number of characters in the text: 163835


In [23]:
print(system_prompt)

Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Include specific types (e.g., cars, trains, bicycles) as well as broader categories (e.g., public transportation).
Provide the terms in Latvian, preserving their original transcription variants as they appear in the text. Text:


In [24]:
# let's use text as a prompt
response = openai_chat(text)
print(response)

The provided text mentions the following land transportation vehicles in Latvian:

* **kamanas** (sleds)
* **tramvaja vagons** (tram)
* **ormanis** (sleigh, specifically referring to a horse-drawn sleigh in this context)
* **kamanu slieces** (sled runners)
* **dzelzceļa vilciens** (train)


There is no mention of cars or bicycles.  The broader category of "public transportation" is not explicitly mentioned although "tramvajs" falls under this category.



In [29]:
elixir_text = Path(r"..\data\docs\PaulM_ProfS_1051757.txt").read_text(encoding="utf-8")
# how many characters in the text
print(f"Number of characters in the text: {len(elixir_text)}")
# first 200 characters
print(elixir_text[:400])

Number of characters in the text: 188657
title: Profesora Sūnas brīnišķīgais eleksīrs
isPartOf:
creator: Paulockis, Miķelis
dateIssued: 1938
publisher: Senatne
publicationPlace: Rīga
firstPublished:
firstEdition: 1938
dateModified:
copyright:
uri: http://dom.lndb.lv/data/obj/1051757
normalization:
ocrCorrection:
verified:



Profesora Sūnas brīnišķīgais eleksīrs

M. Paulockis

Romāns

IZDEVNIECĪBA „SENĀTNE*, RĪGĀ

Sp. „STAR* Rīgā,
Jumara


In [30]:
response = openai_chat(elixir_text)
print(response)

The provided text mentions the following land transportation vehicles in Latvian:

* auto-limuzīna
* tramvaji
* taksometrs
* autobuss
* vilcienos (plural - trains)
* lokomotīves (plural - locomotives)
* auto (plural - cars)
* elektrovāģis


Note that "automobīļus" (plural - automobiles) appears in the text within a sentence describing how people adapted to a changing world, not as a specific event in the narrative.  Therefore it's not included above, as a direct mention of a vehicle type in function within the story is required to be included in the list.



In [32]:
print(system_prompt)

Please extract a comprehensive list of all mentioned land transportation vehicles from the given texts.
Include specific types (e.g., cars, trains, bicycles) as well as broader categories (e.g., public transportation).
Provide the terms in Latvian, preserving their original transcription variants as they appear in the text. Text:


In [35]:
# # let's try different model - google/gemini-2.0-flash-exp:free
# model = "google/gemini-2.0-flash-exp:free"
# model = "google/gemini-exp-1206:free"

# response = openai_chat(elixir_text, model=model)
# print(response)

In [37]:
model = "openai/gpt-4o-mini"
response = openai_chat(elixir_text, model=model)
print(response)

Based on the provided text, here is a comprehensive list of mentioned land transportation vehicles, both specific types and broader categories in Latvian:

1. **Automobiļi** (Cars)
2. **Tramvaji** (Trams)
3. **Taksometri** (Taxis)
4. **Dzelzceļš** (Railway/Train)
5. **Vilciens** (Train)
6. **Centrālapkures katls** (Central Heating Boiler) - indirectly related as part of infrastructure
7. **Gaisa dzelzceļš** (Aerial Cableway) - mentioned in the context of transportation infrastructure

Broader categories and terms:
8. **Sabiedriskā transporta** (Public Transportation)
9. **Līdzekļi** (Vehicles/Means of Transportation)

This list includes various forms of land transportation cited directly in the text and captures the essence of the broader transportation context as discussed.


In [38]:
system_prompt = """You are an expert on Latvian language and transporation.
Please extract a comprehensive list of all mentioned land transportation vehicles from the given Latvian document.
Include all specific types (e.g., vilciens, tramvajs, divritenis, ormanis) as well as broader categories (e.g., dzelceļš, sabiedriskais transports).
Provide the terms solely in Latvian, preserving their original spelling as they appear in the text. Text: 
"""
print(f"Using model {model}")
response = openai_chat(elixir_text, system_prompt=system_prompt, model=model)
print(response)

Using model openai/gpt-4o-mini
Here is a comprehensive list of the land transportation vehicles mentioned in the provided text:

1. **vilciens** (train)
2. **tramvajs** (tram)
3. **taksometrs** (taxi)
4. **divritenis** (bicycle)
5. **auto** (car)
6. **gaisa dzelzceļš** (aerial tramway)
7. **dzelzceļš** (railway)
   
Broader categories:
1. **sabiedriskais transports** (public transportation) 

Please note that the terms are preserved in their original spelling as they appear in the text.


In [39]:
# let's try with Gemini model
model = "google/gemini-2.0-flash-exp:free"
print(f"Using model {model}")
response = openai_chat(elixir_text, system_prompt=system_prompt, model=model)
print(response)

Using model google/gemini-2.0-flash-exp:free
Okay, here's the comprehensive list of land transportation vehicles mentioned in the provided Latvian text, presented solely in Latvian:

*   sabiedriskais transports
*   auto-limuzīnā
*   taksometram
*   taksometra
*   tramvaja
*   tramvaji
*   dzelzceļpārbrauktuvi
*   dzelzceļš
*   vilciens
*   sliedēm
*   auto



## Author subset selection 

We have some specific authors that we will be performing additional testing on.
These authors have been hand curated by domain expert and it is expected that there should be some relevant information in the text.

In [9]:
author_subset = """PaulM  

LaciV  

GulbA 

FimbK 

SartJ 

NiedAi 

LesiVi 

AustA 

LapiK 

RoziP """	
author_list = author_subset.split()
print(author_list)

['PaulM', 'LaciV', 'GulbA', 'FimbK', 'SartJ', 'NiedAi', 'LesiVi', 'AustA', 'LapiK', 'RoziP']


In [33]:
# let's get works from texts from these authors
# this means that the key starts with one of the authors
author_texts = {key: value for key, value in texts.items() if any([key.startswith(author) for author in author_list])}
# how many texts do we have?
print(f"Number of texts: {len(author_texts)}")

Number of texts: 87


In [36]:
# let's order by length
shortest_texts = dict(sorted(author_texts.items(), key=lambda x: len(x[1])))
# how many texts do we have?
print(f"Number of texts: {len(shortest_texts)}")
# print titles
print(*shortest_texts.keys(), sep="\n")

Number of texts: 87
AustA_KaspG_948026
SartJ_Zieda_1051763
PaulM_VienV_1049499
PaulM_SirdP_1049495
LapiK_ManaD_1051717
PaulM_UzDzi_1049498
PaulM_SalaD_1049496
GulbA_Gaidi_1350352
PaulM_MilaA_1051758
PaulM_PieLi_1053489
PaulM_TrauB_1051760
PaulM_ProfS_1051757
LapiK_DodaU_1046848
PaulM_Likte_1049494
LapiK_AtplL_1049448
PaulM_Elks_1049497
PaulM_DejoN_1049493
LapiK_StudF_1293573
PaulM_PasaG_1051778
PaulM_VecaB_1051729
GulbA_JaunV_1053680
LapiK_RevoM_1049451
NiedAi_Salna_1049440
RoziP_DivaS_1053490
GulbA_FantM_1053545
RoziP_UgunC_1046826
LapiK_TajoT_1049517
LaciV_AtbrZ_1051755
LesiVi_LiktR_1051787
LaciV_PulaE_1053689
NiedAi_CilvA_1025449
NiedAi_GrekA_1051756
NiedAi_Sieva_1051739
NiedAi_PiekB_1049492
FimbK_TiltP_1049479
SartJ_Pagar_1047279
NiedAi_SarkV_956032
GulbA_DruvU_1046871
LapiK_NemiU_1293516
NiedAi_AnnaD_1049490
SartJ_Mara_1051666
SartJ_FabrM_1296786
FimbK_KadNa_1049450
LaciV_PiecP_1051752
LapiK_RepuG_1049513
LaciV_PutnB_1053691
LaciV_RotaV_1051746
SartJ_DruvS_1051773
LapiK_Sabru_1046

In [38]:
# let's go through the shortest texts and create a new dictionary with following restrictions
# we want to get two shortest texts from each author
# if some author only has one text we keep that text
# we want to get the shortest text from each author
# we want to get the second shortest text from each author if possible
shortest_texts_2 = {}
# authors are the first part of key before _
authors_work_count_dict = {}
for key, value in shortest_texts.items():
  author = key.split("_")[0]
  if author in authors_work_count_dict:
    authors_work_count_dict[author] += 1
  else:
    authors_work_count_dict[author] = 1
  if authors_work_count_dict[author] <= 2:
    shortest_texts_2[key] = value   
# how many texts do we have?
print(f"Number of texts: {len(shortest_texts_2)}")
# print titles
print(*shortest_texts_2.keys(), sep="\n")

Number of texts: 20
AustA_KaspG_948026
SartJ_Zieda_1051763
PaulM_VienV_1049499
PaulM_SirdP_1049495
LapiK_ManaD_1051717
GulbA_Gaidi_1350352
LapiK_DodaU_1046848
GulbA_JaunV_1053680
NiedAi_Salna_1049440
RoziP_DivaS_1053490
RoziP_UgunC_1046826
LaciV_AtbrZ_1051755
LesiVi_LiktR_1051787
LaciV_PulaE_1053689
NiedAi_CilvA_1025449
FimbK_TiltP_1049479
SartJ_Pagar_1047279
FimbK_KadNa_1049450
LesiVi_Uzvar_1350343
AustA_Puisk_1047362


## Testing land transportation prompts

In [39]:

land_prompt = """You are an expert on Latvian language and land transportation.  
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.  
Do not include any additional information — include only the land transportation terms.
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription.
Show the results in a single list with each unique term on a new line.
Latvian text follows: """
model = "google/gemini-flash-1.5"

In [32]:
first_text = list(author_texts.values())[0]
first_title = list(author_texts.keys())[0]
print(first_title)
# how long is our first text
print(f"Length of first text: {len(first_text)}")
# how many words do we have in the first text
print(f"Number of words in the first text: {len(first_text.split())}")
response = openai_chat(first_text, system_prompt=land_prompt, model=model)
# let's make a folder for keeping these temporary responses
# it will be under data folder
# it will be under responses
# the folder name will be temp_responses with date at the end
now = datetime.now()
temp_responses_folder = Path(f"../data/responses/temp_responses_{now.strftime('%Y_%m_%d')}")
# create folder if it does not exist
temp_responses_folder.mkdir(exist_ok=True)
# assert folder exists
assert temp_responses_folder.exists(), f"Folder {temp_responses_folder} does not exist"
# save response to text file using utf-8 encoding
# use following format for filename: first_title_model_datetime.csv
model_name = model.replace("/", "_")
model_name = model_name.replace(":", "_")
filename = f"{first_title}_{model_name}_{now.strftime('%Y_%m_%d_%H_%M_%S')}.csv"
# save response to file
# we want to also count how many times each word in response appears in the text
# so we go through response line by line and count how many times each word appears in the text
response_list = []
for line in response.split("\n"):
  # if line is empty continue
  if not line.strip():
    continue
  response_list.append((line, first_text.count(line)))
  # print current response and count
  print(f"{line}: {first_text.count(line)}")
with open(temp_responses_folder / filename, "w", encoding="utf-8") as f:
  # write head
  # response, count
  f.write("response,count\n")
  for line, count in response_list:
    f.write(f"{line},{count}\n")
  # write prompt to file in a new line
  f.write(f"\n{land_prompt},0\n")
print(response)

AustA_GaraJ_1025406
Length of first text: 1589329
Number of words in the first text: 235026
Using model google/gemini-flash-1.5
System prompt: You are an expert on Latvian language and land transportation.  
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.  
Do not include any additional information — include only the land transportation terms.
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription.
Show the results in a single list with each unique term on a new line.
Latvian text follows: 
Start time: 2025-02-26 15:31:16.212298
Time taken: 0:00:24.401545
gājieni: 3
streiki: 1
demonstrācijas: 0
bēgšana: 4
karatavas: 1
pajūgs: 2
ratiņi: 7
kamanas: 4
karietes: 5
vilciens: 17
ātrvilciens: 1
zirgs: 34
ormaņi: 1
važonis: 1
kuģis: 6
kamanas: 4
ragavas: 2
tramvajs: 2
velosipēds: 1
automobilis: 2
ķerre: 0
orē: 17
rateļos: 2
droška: 8
kamanās: 12
gājieni
streiki
demonstrācijas
bēgš

In [40]:
responses_folder

WindowsPath('../data/responses')

In [42]:
# let's call the prompt all 20 shortest texts
# we will use current gemini model
openai_chat_all(texts=shortest_texts_2, 
                system_prompt=land_prompt, 
                verbose=True, 
                delay=0.2, 
                max_retries=10, 
                model=model, 
                responses_folder=responses_folder)

Subfolder: ..\data\responses\2025_02_26_google_gemini-flash-1.5_with_terms


  0%|          | 0/20 [00:00<?, ?it/s]

Processing key: AustA_KaspG_948026
Using model google/gemini-flash-1.5
System prompt: You are an expert on Latvian language and land transportation.  
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.  
Do not include any additional information — include only the land transportation terms.
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription.
Show the results in a single list with each unique term on a new line.
Latvian text follows: 
Start time: 2025-02-26 15:45:16.500315
Time taken: 0:00:04.943359


  5%|▌         | 1/20 [00:06<01:54,  6.02s/it]

Processing key: SartJ_Zieda_1051763
Using model google/gemini-flash-1.5
System prompt: You are an expert on Latvian language and land transportation.  
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.  
Do not include any additional information — include only the land transportation terms.
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription.
Show the results in a single list with each unique term on a new line.
Latvian text follows: 
Start time: 2025-02-26 15:45:22.456995
Time taken: 0:00:03.853282


...

 90%|█████████ | 18/20 [01:56<00:14,  7.49s/it]

Time taken: 0:00:07.197240
Processing key: LesiVi_Uzvar_1350343
Using model google/gemini-flash-1.5
System prompt: You are an expert on Latvian language and land transportation.  
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.  
Do not include any additional information — include only the land transportation terms.
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription.
Show the results in a single list with each unique term on a new line.
Latvian text follows: 
Start time: 2025-02-26 15:47:12.800269
Time taken: 0:00:11.755481


 95%|█████████▌| 19/20 [02:09<00:09,  9.08s/it]

Processing key: AustA_Puisk_1047362
Using model google/gemini-flash-1.5
System prompt: You are an expert on Latvian language and land transportation.  
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.  
Do not include any additional information — include only the land transportation terms.
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription.
Show the results in a single list with each unique term on a new line.
Latvian text follows: 
Start time: 2025-02-26 15:47:25.535258
Time taken: 0:00:09.909806


100%|██████████| 20/20 [02:20<00:00,  7.00s/it]


In [43]:
land_prompt_2 = """You are an expert on Latvian language and land transportation.   
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.   
Include: Any specific vehicles used for land transport (e.g., cars, buses, bicycles, motorcycles, trains, trams, sleighs, horse carriages, etc.). 
Exclude: Any vehicles that are not used for land transportation (e.g., airplanes, hot air balloons, boats, ships, etc.). 
Do not include any additional information — include only the land transportation terms. 
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription. 
Show the results in a single list with each unique term on a new line. 
Latvian text follows: """

In [44]:
# run through all shortest texts with land_prompt_2
# we use suffix land_prompt_2
openai_chat_all(texts=shortest_texts_2, 
                system_prompt=land_prompt_2, 
                verbose=True, 
                delay=0.2, 
                max_retries=10, 
                model=model, 
                responses_folder=responses_folder, 
                responses_folder_suffix="land_prompt_2")

Subfolder: ..\data\responses\2025_02_26_google_gemini-flash-1.5_land_prompt_2


  0%|          | 0/20 [00:00<?, ?it/s]

Processing key: AustA_KaspG_948026
Using model google/gemini-flash-1.5
System prompt: You are an expert on Latvian language and land transportation.   
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.   
Include: Any specific vehicles used for land transport (e.g., cars, buses, bicycles, motorcycles, trains, trams, sleighs, horse carriages, etc.). 
Exclude: Any vehicles that are not used for land transportation (e.g., airplanes, hot air balloons, boats, ships, etc.). 
Do not include any additional information — include only the land transportation terms. 
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription. 
Show the results in a single list with each unique term on a new line. 
Latvian text follows: 
Start time: 2025-02-26 15:54:04.435254
Time taken: 0:00:06.134803


  5%|▌         | 1/20 [00:07<02:16,  7.16s/it]

Processing key: SartJ_Zieda_1051763
Using model google/gemini-flash-1.5
System prompt: You are an expert on Latvian language and land transportation.   
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.   
Include: Any specific vehicles used for land transport (e.g., cars, buses, bicycles, motorcycles, trains, trams, sleighs, horse carriages, etc.). 
Exclude: Any vehicles that are not used for land transportation (e.g., airplanes, hot air balloons, boats, ships, etc.). 
Do not include any additional information — include only the land transportation terms. 
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription. 
Show the results in a single list with each unique term on a new line. 
Latvian text follows: 
Start time: 2025-02-26 15:54:11.593226
Time taken: 0:00:04.484976


...

 90%|█████████ | 18/20 [02:07<00:17,  8.98s/it]

Processing key: LesiVi_Uzvar_1350343
Using model google/gemini-flash-1.5
System prompt: You are an expert on Latvian language and land transportation.   
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.   
Include: Any specific vehicles used for land transport (e.g., cars, buses, bicycles, motorcycles, trains, trams, sleighs, horse carriages, etc.). 
Exclude: Any vehicles that are not used for land transportation (e.g., airplanes, hot air balloons, boats, ships, etc.). 
Do not include any additional information — include only the land transportation terms. 
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription. 
Show the results in a single list with each unique term on a new line. 
Latvian text follows: 
Start time: 2025-02-26 15:56:11.674954


 95%|█████████▌| 19/20 [02:15<00:08,  8.79s/it]

Time taken: 0:00:07.343140
Processing key: AustA_Puisk_1047362
Using model google/gemini-flash-1.5
System prompt: You are an expert on Latvian language and land transportation.   
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.   
Include: Any specific vehicles used for land transport (e.g., cars, buses, bicycles, motorcycles, trains, trams, sleighs, horse carriages, etc.). 
Exclude: Any vehicles that are not used for land transportation (e.g., airplanes, hot air balloons, boats, ships, etc.). 
Do not include any additional information — include only the land transportation terms. 
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription. 
Show the results in a single list with each unique term on a new line. 
Latvian text follows: 
Start time: 2025-02-26 15:56:20.041984
Time taken: 0:00:42.928757


100%|██████████| 20/20 [02:59<00:00,  8.98s/it]


## Testing OpenAI models

In [45]:
model = "openai/gpt-4o-2024-11-20"
print(f"Using model {model}")

Using model openai/gpt-4o-2024-11-20


In [47]:
# last entry in shortest_texts_2
last_entry = list(shortest_texts_2.values())[-1]
# last entry title
last_entry_title = list(shortest_texts_2.keys())[-1]
print(last_entry_title)
# how many words?
print(f"Number of words: {len(last_entry.split())}")

AustA_Puisk_1047362
Number of words: 85925


In [48]:
# since OpenAI models have 128k token limit we need to split the text that is over 40k words long into smaller parts - chunks
# let's write a function that given a text name and text, and max chunk size will split the text into chunks
# we will use . or ! or ? as sentence separator to split the text into aproximately equal chunks
# if the chunk is too big we will split it into smaller chunks
# we will return a dictionary with key as text_name_chunk_number and value as chunk text
def get_chunks(text_name, text, max_chunk_size=40000, split_chars=""):
    # split text into sentences
    # if split_chars is empty we will use whitespace as separator
    if not split_chars:
        tokens = text.split()
    else:
        tokens = text.split(split_chars)
    # initialize chunk list
    chunk_dict = {}
    for i in range(0, len(tokens), max_chunk_size):
        chunk = tokens[i:i+max_chunk_size]
        chunk_dict[f"{text_name}_{i//max_chunk_size}"] = " ".join(chunk)
    return chunk_dict

# let's test it on last text which is the longest
long_dict = get_chunks(last_entry_title, last_entry)
# how many chunks do we have?
print(f"Number of chunks: {len(long_dict)}")

Number of chunks: 3


In [49]:
# let's go through our shortest_texts_2 and create chunked_dictionary
chunked_dict = {}
for key, value in shortest_texts_2.items():
    chunked_dict.update(get_chunks(key, value))
# how many chunks do we have?
print(f"Number of chunks: {len(chunked_dict)}")

Number of chunks: 31


In [50]:
# now let's run our chunked_dict through openai_chat_all
# we will use land_prompt_2
# we will skip verbose
# we will use delay 0.2
# we will use max_retries 10
# we will use current OpenAI model
# we will use responses_folder
# we will use with land_prompt_2 suffix
openai_chat_all(chunked_dict, land_prompt_2, False, 0.2, 10, model, responses_folder, "land_prompt_2")

100%|██████████| 31/31 [05:32<00:00, 10.72s/it]


### Basic prompt through OpenAI

In [51]:
land_prompt = """You are an expert on Latvian language and land transportation.  
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.  
Do not include any additional information — include only the land transportation terms.
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription.
Show the results in a single list with each unique term on a new line.
Latvian text follows: """

In [52]:
# now let's run land_prompt through openai_chat_all
# we will use chunked_dict
# we will use land_prompt
# we will skip verbose
# we will use delay 0.2
# we will use max_retries 10
# we will use current OpenAI model
# we will use responses_folder
# we will use with land_prompt suffix
openai_chat_all(chunked_dict, land_prompt, False, 0.2, 10, model, responses_folder, "land_prompt")

100%|██████████| 31/31 [06:07<00:00, 11.86s/it]


## Gemini Flash 2.0

In [53]:
# let's use the latest full Gemini Flash model
model = "google/gemini-2.0-flash-001"
print(f"Using model {model}")

Using model google/gemini-2.0-flash-001


In [55]:
print(land_prompt)

You are an expert on Latvian language and land transportation.  
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.  
Do not include any additional information — include only the land transportation terms.
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription.
Show the results in a single list with each unique term on a new line.
Latvian text follows: 


In [56]:
print(land_prompt_2)    

You are an expert on Latvian language and land transportation.   
Extract a comprehensive list of all mentioned land transportation vehicles from the given texts.   
Include: Any specific vehicles used for land transport (e.g., cars, buses, bicycles, motorcycles, trains, trams, sleighs, horse carriages, etc.). 
Exclude: Any vehicles that are not used for land transportation (e.g., airplanes, hot air balloons, boats, ships, etc.). 
Do not include any additional information — include only the land transportation terms. 
List just the terms in Latvian, exactly as they appear in the text, preserving their original spelling and transcription. 
Show the results in a single list with each unique term on a new line. 
Latvian text follows: 


In [57]:
# let's use land_prompt on shortest_texts_2
openai_chat_all(shortest_texts_2, land_prompt, False, 0.2, 10, model, responses_folder, "land_prompt")

100%|██████████| 20/20 [01:53<00:00,  5.68s/it]
